In [1]:
import pandas as pd

from config import PATH_URLS, N_URLS_TO_FETCH
N_URLS_TO_FETCH

-1

In [2]:
URLS = pd.read_csv(PATH_URLS).head(N_URLS_TO_FETCH)
URLS

,pws_id,pws_url_id,url
0,CA0103040,3,https://sdwis.waterboards.ca.gov/PDWW/JSP/Wate...
1,CA0103041,4,https://sdwis.waterboards.ca.gov/PDWW/JSP/Wate...
2,CA0105002,15,https://sdwis.waterboards.ca.gov/PDWW/JSP/Wate...
3,CA0105003,16,https://sdwis.waterboards.ca.gov/PDWW/JSP/Wate...
4,CA0105008,18,https://sdwis.waterboards.ca.gov/PDWW/JSP/Wate...
5,CA0105009,19,https://sdwis.waterboards.ca.gov/PDWW/JSP/Wate...
6,CA0105010,20,https://sdwis.waterboards.ca.gov/PDWW/JSP/Wate...
7,CA0105012,22,https://sdwis.waterboards.ca.gov/PDWW/JSP/Wate...
8,CA0105013,23,https://sdwis.waterboards.ca.gov/PDWW/JSP/Wate...
9,CA0105016,26,https://sdwis.waterboards.ca.gov/PDWW/JSP/Wate...


# Fetching the HTML pages

To get the data from the server, we need to issue a request for each WSD URL. The number is not enormous but at the same time non-negligible. since, in general, we want to throttle (limit the rate) our requests since we don't know if this will trigger some security response from the server, e.g. refusing connections. 

In addition, it's useful (especially when scraping) to be able to cache the results, so that we fetch the data once and can process it without limitations afterwards.

In [3]:
import requests
import time

def _make_throttle_hook(timeout=1):
    """
    Returns a response hook function which sleeps for `timeout` seconds.
    """
    def hook(response, *args, **kwargs):
        time.sleep(timeout)
        return response

    return hook

def get_session(session=None, rate_max=.5, timeout=None, path_cache_db=''):
    """
    `rate_max` is in requests per second; `timeout` is in seconds.
    """
    session = session or requests.Session()
    timeout = timeout or 1 / rate_max

    session.hooks = {'response': _make_throttle_hook(timeout)}
    return session

In [4]:
SESSION = get_session(timeout=.1)
SESSION

In [5]:
from tqdm._tqdm_notebook import tqdm_notebook

from config import PATH_HTML_PAGES

In [6]:
PATH_HTML_PAGES

PosixPath('data/intermediate-results/html-pages')

In [7]:
def process_row(row, session, path_base):
    status = {}

    pws_id = row.pws_id
    url = row.url
    
    status.update({'PWSID': pws_id})
    
    path = (path_base / str(pws_id)).with_suffix('.html')
    
    if path.exists():
        if path.stat().st_size > 0:
            status.update(source=path)
    else:
        resp = session.get(url)
        if resp:
            path.write_text(resp.text)
        else:
            print(f'{url}: {resp}')
        status.update(source=url)
        
    return status

def fetch_urls(_df_urls, session, path_base=None):
    df = _df_urls.reset_index()

    with tqdm_notebook(total=len(df)) as progress_bar:
        for row in df.itertuples():
            
            status = process_row(row, session, path_base)
            
            progress_bar.set_postfix(status)
            progress_bar.update(1)

In [9]:
fetch_urls(URLS, SESSION, path_base=PATH_HTML_PAGES)